<a href="https://colab.research.google.com/github/Conor-Luppnow/Walt-Whitman_Poem_Gen/blob/master/FinalProject4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Code to install necessary libraries and perform authorization

"""
When you run this code, you will be asked to authenticate which can be done by 
clicking the click outputed and copy pasting the access code. Sometime you have to do this couple of time to gain access

If you ran this code after successful authentication during your session,
when asked for verification code without any link to browser, enter your google password
"""

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144429 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.21-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
## Mount google drive 

!mkdir -p drive 
!google-drive-ocamlfuse drive

In [0]:
## Create a folder called "testing"(any name you want )

## Setting the working directory
import os
os.chdir("drive/Colab_Notebooks/Project4whitman")

In [4]:
!pip install tensorflow-gpu==1.15

     |████████████████████████████████| 411.5MB 37kB/s 
     |████████████████████████████████| 512kB 50.2MB/s 
     |████████████████████████████████| 3.8MB 46.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=c0db4db0a892be210dbe7e1850fd61d3632c1fd04e819c4f4007a149ffdd86c7
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc4 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc4 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0rc4 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.

In [5]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.utils import np_utils
import tensorflow as tf
#import needed libraries

Using TensorFlow backend.


In [0]:

#acces the whitman text, and make all the letters lowecase
file = "whitman.txt"
text = open(file, 'r', encoding='utf-8').read()
text = text.lower()

In [7]:

#get the number of characters
chars = sorted(list(set(text)))
#map characters to ints
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

num_chars = len(text)
num_vocab = len(chars)
print ("Total Characters: " +str(num_chars))#total amount of characters in text file
print ("Unique Characters: " + str(num_vocab))#total amount of unique charactres

Total Characters: 736887
Unique Characters: 56


In [15]:
seq_length = 100
X = [] # create dataset 
Y = []
for i in range(0, num_chars - seq_length, 1):
	s_in = text[i:i + seq_length]
	s_out = text[i + seq_length]
	X.append([char_to_int[char] for char in s_in])
	Y.append(char_to_int[s_out])
num_patterns = len(X)
print ("Total Patterns: " + str(num_patterns))


Total Patterns: 736787


In [0]:
from keras.utils.np_utils import to_categorical

In [0]:
X_modified = np.reshape(X,(num_patterns,seq_length,1))#reshape dataset
X_modified = X_modified/float(num_vocab)
Y_modified = to_categorical(Y)#for use in categorical_crossentropy.

In [0]:
from keras.layers import Dense,Dropout,LSTM

In [20]:
X_modified.shape

(736787, 100, 1)

In [21]:
Y_modified.shape

(736787, 56)

In [0]:
#create LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.1))
model.add(Dense(Y_modified.shape[1], activation='softmax'))
#use categorial crossentropy loss
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
from keras.callbacks import ModelCheckpoint
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [29]:
# Now time to fit our input to model
model.fit(X_modified,Y_modified,epochs=40,batch_size=128,callbacks=callbacks_list)


Epoch 1/40
  6656/736787 [..............................] - ETA: 19:27 - loss: 1.3641

KeyboardInterrupt: ignored

In [0]:

weight_name = "weights-improvement-14-1.4123.hdf5"
model.load_weights(weight_name)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
def sample(preds, temperature=.5):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [31]:
Y_modified.flatten()


array([0., 1., 0., ..., 0., 0., 0.], dtype=float32)

In [32]:

# pick a random seed
start = np.random.randint(0, len(X)-1)
pattern = X[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")  
# generate characters
a=""
for i in range(400):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(num_vocab)
    prediction = model.predict(x, verbose=0)
    prediction = np.reshape(prediction, prediction.size)
    #index = np.random.choice(Y_modified.shape[1], 1, p=prediction[0])[0]
    index = sample(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]	
    a = a+(str(result))
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print(a + '\n')
print ("End")

Seed:
" 

o me! o life!

  o me! o life! of the questions of these recurring,
  of the endless trains of the "


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


 city of the west,
  the ships and the sea showld,
  the face of the soul of the contummtion of the rest,
  the form continually and jase of the louer and beautiful and his,
  the sciool with the singers and north and shall be stole the singers,
  the face of the old all the sea, the slow and shapes of the present,
  the siee for the suree the stars and the ereshes, and the same and
      the face

End
